# Downloading Experiments

This notebook is a proof of concept

Right now, we have some flaws in our raw data download system. We want to make sure we get this right because we will be downloading and processing hundreds or thousands of GBs of raw data parquet files.

So, we'll be messing around here with some implementations, and if they work, we'll be replacing parts of our original code with this.

In [4]:
%pip install huggingface_hub --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Getting file names

We want to use file names and other meta data to do dupe checks of what we've already processed, and see what we still need to download from a certain month. Let's run this code that gets all file names from the repo, to see what the data looks like.

In [8]:
# Here we get a list of raw data files for the given month/year

# File names in the remote repo are structured like:
# data/year=2025/month=03/train-00001-of-00065.parquet
# Obviously, there will be different amounts of them so it won't always be -00065.parquet

from huggingface_hub import HfApi
from pathlib import Path

year = "2025"
month = "03"  # <-- type manually

api = HfApi()
files = api.list_repo_files(
    repo_id="Lichess/standard-chess-games",
    repo_type="dataset"
)

# Filter for that year/month
target_prefix = f"data/year={year}/month={month}/"
all_file_names_in_month = [f for f in files if f.startswith(target_prefix)]

print(len(all_file_names_in_month), f"files found for {year}-{month}")
for f in all_file_names_in_month[:20]:  # preview first 20
    print(f)

69 files found for 2025-03
data/year=2025/month=03/train-00000-of-00069.parquet
data/year=2025/month=03/train-00001-of-00069.parquet
data/year=2025/month=03/train-00002-of-00069.parquet
data/year=2025/month=03/train-00003-of-00069.parquet
data/year=2025/month=03/train-00004-of-00069.parquet
data/year=2025/month=03/train-00005-of-00069.parquet
data/year=2025/month=03/train-00006-of-00069.parquet
data/year=2025/month=03/train-00007-of-00069.parquet
data/year=2025/month=03/train-00008-of-00069.parquet
data/year=2025/month=03/train-00009-of-00069.parquet
data/year=2025/month=03/train-00010-of-00069.parquet
data/year=2025/month=03/train-00011-of-00069.parquet
data/year=2025/month=03/train-00012-of-00069.parquet
data/year=2025/month=03/train-00013-of-00069.parquet
data/year=2025/month=03/train-00014-of-00069.parquet
data/year=2025/month=03/train-00015-of-00069.parquet
data/year=2025/month=03/train-00016-of-00069.parquet
data/year=2025/month=03/train-00017-of-00069.parquet
data/year=2025/mont

## Dupe checks

Now that we have the list of raw data file names for the given month and year, we'll perform our dupe checks. This parses through the list of file names to make sure we haven't already processed any of these files.

Each file is a 1GB download, so it's obviously in our best interest not to download a file we've already processed.

In [29]:
# Parse the list of raw data file names to make sure we haven't already processed any of these files, and skip downloading any dupes

from huggingface_hub import get_hf_file_metadata, hf_hub_url

import sys
from pathlib import Path

# Current working directory (should be project root)
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from utils.file_processing.raw_data_file_dupe_checks import FileRegistry  # noqa: E402

# Init registry
registry = FileRegistry()

# Remove files already processed
non_dupe_files = []
for f in all_file_names_in_month:
    url = hf_hub_url(
        repo_id="Lichess/standard-chess-games",
        repo_type="dataset",
        filename=f,
    )
    meta = get_hf_file_metadata(url=url)
    size = meta.size
    etag = meta.etag
    if not registry.is_file_processed(f"{year}-{month}", Path(f).name, size, etag):
        non_dupe_files.append(f)

print(len(non_dupe_files), "new files to download")

69 new files to download


In [30]:

from huggingface_hub import hf_hub_download
from pathlib import Path
import shutil

year, month = 2025, 3
max_files_to_download = 1  # for testing; set to None to download all
file_names_to_download = all_file_names_in_month[:max_files_to_download]

local_dir = Path("../data/raw/better_downloading_experiments")
local_dir.mkdir(parents=True, exist_ok=True)

for file_to_download in file_names_to_download:
    # Download (HF handles caching)
    downloaded_path = hf_hub_download(
        repo_id="Lichess/standard-chess-games",
        repo_type="dataset",
        filename=file_to_download,
    )

    # Rename with prefix and move to flat folder
    # HF sends it to me in a nested file structure; but we don't want that so here we reoganize it to the file structure we want locally

    # Prefixing file name with year and month
    # The file will be deleted immediately after processing, but this could still be helpful for debugging
    target_filename = f"{year}-{month:02d}-{Path(file_to_download).name}"
    target_path = local_dir / target_filename
    shutil.copy(
        downloaded_path, target_path
    )  # or move() if you want to remove original
    print(f"File saved to {target_path}")

File saved to ../data/raw/better_downloading_experiments/2025-03-train-00000-of-00069.parquet
